In [6]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [7]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory+automation' # topic of the search
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(5) # first 5 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Request failed with status code: 401
{"httpStatus":403,"description":"Forbidden"}


# Answering questions on the results with OpenAI GPT

In [8]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

       For the problem you found, please only output the following: 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 250-300 words)
        - "User group": Which user group is impacted by this problem the most (please write 20-50 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 80-100 words)
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

In [9]:
# second agent: 

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria.
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)
        
    Be very stringent with your ratings:
        1: Does not meet expectations
        2: Meets expectations adequately
        3: Exceeds expectations significantly
    
    A rating of 3 should be reserved for exceptional performance in the given criterion.

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 50 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


In [10]:
# third agent to give best problem 

user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
   Please output the problem with the highest overall rating in the {rated_problems_list}, including the following bullet points: 
       - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 100-150 words)
        - "User group": Which user group is impacted by this problem the most (please write 10-30 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 40-50 words)
 
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print(completion.choices[0].message.content)
print("\n---\n")




OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
problems_and_ratings = [a + b for a, b in zip(problems_list, rated_problems_list)]

def print_menu(options):
    for idx, option in enumerate(options, start=1):
        print(f"{idx}. {option}")

def get_user_choice(num_options):
    while True:
        try:
            choice = int(input("Enter your choice (number): "))
            if 1 <= choice <= num_options:
                return choice
            else:
                print(f"Please enter a number between 1 and {num_options}.")
        except ValueError:
            print("Please enter a valid integer.")

def main():
    options = problems_and_ratings
    print_menu(options)
    choice = get_user_choice(len(options))
    print(f"You chose: {options[choice - 1]}")

if __name__ == "__main__":
    main()

1. - **Title**: Adaptive Production Line Layouts

- **Explanation**: As factory automation evolves, one area ripe for innovation within the adjacent possible is the development of adaptive production line layouts. Traditional production lines are often static, designed for mass production of a single product type or variants thereof with minimal changes. However, consumer behavior shifts towards customization and the demand for a wider variety of products in smaller quantities challenge this model. Advances in robotics, AI, and modular design technologies present an opportunity to reimagine factory floors as dynamic environments. These adaptive layouts could reconfigure in real-time or near-real-time to accommodate different products, reflecting changes in consumer demand or to optimize for new manufacturing processes introduced by technological advancements. This capability would require sophisticated software for planning and simulation, advanced sensors and IoT devices for real-time

# Getting data from Mergeflow's API

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'home appliances' # topic is home appliances
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(20) # first 20 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 20 documents.


In [ ]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Land, a Cleveland-based company specializing in electric mobility and portable battery manufacturing, recently secured $3 million in funding. Founded in 2020 by Scott Colosimo, Land focuses on the importance of energy continuity and mobility in modern life. The company offers an e-Moto ecosystem with a portable power platform called CORE, which features innovative technology enabling users to distribute energy from the battery for various devices both on and off the grid. Through their 'Power Tap' accessory, consumers can charge devices like phones, laptops, and household appliances using the electric vehicle's USB-C port and standard plug.

The undisclosed backers of Land's recent funding round have facilitated the company's growth trajectory. In a previous Series A funding round in September 2023, Land raised $7 million. This financial injection was instrumental in upgrading their 30,000-square-foot manufacturing facility in Cleveland and expanding their team. As a result, Land was a

SPAN, a leading innovator in home electrification technology, has secured $96 million in funding to drive product innovation and accelerate their mission of electrifying 10 million homes by 2030. This Series B2 fundraise, led by Wellington Management and supported by various investors such as Congruent Ventures and Qualcomm Ventures, brings SPAN's total funding to $231 million. The company plans to expand its ecosystem for whole-home energy management, enhance its partnerships with major home appliance brands like Kenmore and solar providers like Sunrun, and integrate its technology to manage high-energy loads in homes.

With a focus on research and development, SPAN is establishing a cutting-edge facility near its headquarters in San Francisco to facilitate advanced product testing in various grid scenarios. Additionally, the company is investing in talent acquisition, particularly in software and hardware engineering, to bolster capabilities in system modeling, forecasting, DER integ

Einride, a freight mobility technology company based in Stockholm, Sweden, recently secured $500 million in funding through a combination of debt and Series C financing. This funding round comprised a $300 million debt facility and a $200 million equity contribution, which included a $90 million convertible note raised earlier in the year. The debt financing was led by Barclays Europe, while the equity investors included notable names such as Swedish pension fund AMF, EQT Ventures, Northzone, Polar Structure, Norrsken VC, and Temasek, among others.

Under the leadership of Robert Falck, the Founder and CEO, Einride focuses on designing, developing, and implementing technologies for freight mobility. Over the past year, the company has expanded its operations into several European countries, including Germany, Belgium, Netherlands, Luxembourg, and Norway. Einride has also onboarded new clients such as Electrolux, GE Appliances (a Haier company), and Bridgestone.

A significant milestone

Wikifactory, a Denmark-based online platform, recently secured $2.5 million in pre-Series A funding. This funding round saw participation from both existing shareholders and new investors, including Lars Seier Christensen's investment vehicle, bringing the total funding raised by the company to nearly $8 million. Led by Nicolai Peitersen, Wikifactory aims to facilitate collaboration among developers, designers, engineers, and startups worldwide to create hardware solutions that address real-world problems in real-time.

The company's core mission is to establish the Internet of Production, a system that connects product definitions, software services, and manufacturing solutions based on open standards. Currently, over 130,000 product developers from 190 countries utilize Wikifactory to work on a diverse range of projects, including robotics, electric vehicles, drones, agri-tech, sustainable energy appliances, lab equipment, 3D printers, smart furniture, biotech fashion materials, and 

Sun King, a company based in Nairobi, Kenya, specializing in providing solar energy products for off-grid homes in Africa and Asia, recently secured $260 million in Series D funding. The funding round was led by BeyondNetZero, the climate investing venture of General Atlantic, in collaboration with M&G Investments' Catalyst team and Arch Emerging Markets Partners. Out of the total funding, $100 million was designated for primary investment, while the founders of Sun King will retain voting control of the board.

The company plans to utilize the funds for several purposes, including expanding geographically and diversifying its product line. The expansion strategy involves offering larger solar systems with AC-electricity inverters capable of powering appliances like refrigerators, in addition to introducing new products such as mobile phones. Founded in 2007 by CEO Patrick Walsh as Greenlight Planet, Sun King operates as an off-grid solar energy enterprise, providing energy solutions t

In [ ]:
Problem = problems_list[choice - 1]

print(Problem)

Title: Sustainable Water Appliance Integration

Explanation: As consumer awareness and regulatory requirements around sustainability increase, there's a growing demand for home appliances that not only reduce water and energy consumption but also integrate seamlessly with home water treatment systems like softeners and purifiers. The problem lies in the lack of interconnected appliances that can communicate with water treatment systems to optimize their operation based on water quality. For instance, washing machines and dishwashers that adjust their cycles and detergent usage based on real-time feedback from a connected water softener system could significantly enhance efficiency and sustainability. This innovation requires a blend of IoT technology, advances in material science for developing more efficient filters, and AI for predictive analytics and optimization.

User group: Environmentally-conscious homeowners

Pain point: This group struggles with the inefficiency and increased 

# Answering questions on the results with OpenAI GPT

In [ ]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the Walt Disney Method: 
        It is a tool for brainstorming ideas. It involves using 3 sequential roles, namely the Dreamer, the Realist, and the Critic.
        The Dreamer: you focus on free association and brainstorming of ideas. Anything goes here. It's your chance to let your imagination run wild! Be enthusiastic and approach the task with questions such as “What would be nice?”, “What would be the optimal situation?” or “What would I wish for?” Assume that everything is possible.
        The Realist: Be realistic and decide which of your ideas are actually possible or practical. What would you need to do in the real world in order to make them happen? Take a pragmatic approach and ask which materials, which knowledge, which techniques there are with which the thoughts of the dreamer can be realised.
        The Critic: Examine proposals and its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. It is about identifying opportunities and risks, pointing out limits and gaps, and expressing constructive and always positive criticism. Try to identify possible sources of error.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
dreamer_solutions = {}
i = 0

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Imagine you are the Dreamer. 
    
    Take a deep breath and work on this problem step by step. 

    
    Please develop 5 really specific ideas/visions on how this problem can be solved. The solutions should consider the following points:
    - You can let your imagination run wild and you do not care about possible limitations. It's more important the idea is new then it can really work.  
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    The solution should contain the following contents:
    1) Your solution in max 3 words (title)
    2) Your solution summarized in 3 sentences.
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) Why the solution is kind of new (not existing on the market)
    5) Please provide some sources of inspiration that you used from the document excerpts.


    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
dreamer_solutions[i] = completion.choices[0].message
i += 1


#Realist prompt (idea: evaluate if it is possible and what would be needed)
realist_solutions = {}

user_prompt1 = f"""

    Imagine you are the Realist. 
    
    Take a deep breath and work on this problem step by step. 

    
    Consider the following solutions: {dreamer_solutions}.
    
    Please give me the 3 clearly speperated pieces of information for each of the solutions (in total 50 words max):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 1 detail": What would you need to do in the real world in order to make the solution happen? Answer which knowledge, which techniques there are with which the solutions can be realised. Explain in 5 sentences. 
    4) "Evaluation criteria 1": Is the solution actually possible? Answer can be 1,2 or 3 (1 = not possible, 2 = maybe possible, 3 = possible) (be the most critical person ever. Only give max. 1 of the 3 solution a 3)

    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
realist_solutions[0] = completion.choices[0].message


#Critics prompt (idea: evaluate if there are a lot potential sources of error)
critics_solutions = {}

user_prompt2 = f"""

    Imagine you are the Critic. 
    
    Consider the following solutions: {realist_solutions}.
    
    Take a deep breath and work on this problem step by step. 
    
    Please give me the following 4 clearly seperated pieces of information for each solution (40 words in total):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 2 details": Identify possible sources of error. Look for risks, pointing out limits and gaps, and expressing constructive criticism. Examine its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. (1 sentence only)
    4) "Evaluation criteria 2": Does it have many potential sources of errors? Answer can be 1,2 or 3 (1 = many sources of error, 2 = some sources of error, 3 = few sources of error) (be the most critical person ever. Only give max. 1 of the 3 solution a 3) (1 sentence)
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
critics_solutions[0] = completion.choices[0].message


#Summary

summary_solutions = {}

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas: {dreamer_solutions}. 
    For each idea you considered how feasible it is {realist_solutions}. 
    Last, you evaluated the amount of potential sources of errors: {critics_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (3 words max)
    “Solved Problem”: 1 or 2 sentence summary of the problem that is being solved 
    “Solution”: Provide a summary of the solution and how it works (in 4-5 sentences). Be dilligent on this task and make sure yo use the length limit. 
    “Impact”: How does the solution solve the specific problem (in 2-3 sentences)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution (in 4-5 sentences)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions[0] = completion.choices[0].message



### 1. Water Quality Assistant (WQA)

**Solution Summary**:  
The Water Quality Assistant is an AI-driven smart home assistant that monitors and adjusts water quality in real-time. It integrates with all water-using appliances and treatment systems, optimizing water softness, purity, and temperature based on the specific needs of each appliance. The WQA uses predictive analytics to anticipate water needs, ensuring optimal efficiency and sustainability.

**Customer Journey**:  
Upon installation, the customer sets preferences via a smartphone app. The WQA then continuously monitors water quality, communicating with appliances like washing machines, dishwashers, and purifiers to adjust settings in real-time for optimal performance. Customers receive monthly reports on water usage, savings, and suggestions for further efficiency improvements.

**Novelty**:  
This solution is novel because it introduces an overarching AI that not only adjusts water quality on a granular level for each appl

### 1. Water Quality Assistant (WQA)

**Title**: Water Quality Assistant (WQA)

**Summary**: The WQA is an innovative AI-driven smart home assistant designed to optimize water quality for home appliances in real-time, featuring predictive analytics for water needs and a user-friendly smartphone app for monitoring.

**Evaluation criteria 2 details**: The reliance on AI and the necessity for widespread IoT device compatibility could present significant challenges in terms of privacy, data security, and interoperability across diverse appliance brands.

**Evaluation criteria 2**: 2

### 2. HydroSync Mesh Network

**Title**: HydroSync Mesh Network

**Summary**: HydroSync leverages blockchain technology to optimize water usage and quality across a community, using collective data to enhance distribution and featuring a communal dashboard for promoting environmental awareness.

**Evaluation criteria 2 details**: The requirement for community-wide adoption and the complexity of implementing b

In [ ]:
# Initial rating of the solutions

initial_rating = {}

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. When evaluating the opportunities, you should take the perspective of a potential client of these solutions, active in the specific industry of {query}.
You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Task: Evaluate the provided solutions based on detailed criteria.

Input Text: {summary_solutions}

Instructions:

Extract Solutions: Begin by identifying all solutions within the input text. For each solution, note the following details:
    - Name
    - Solved Problem
    - Solution Description
    - Impact
    - Technology Used
    - Sources
    
Develop Evaluation Criteria: Create 10 specific criteria to assess the solutions. These criteria should enable a clear comparison between the solutions, focusing on aspects such as innovation, scalability, social impact, and cost-effectiveness.

Rate Each Solution: Apply the criteria to each solution, assigning ratings on a scale from 1 to 3 (1=Lowest, 3=Highest). Be stringent with the ratings:
    1: Does not meet expectations
    2: Meets expectations adequately
    3: Exceeds expectations significantly

A rating of 3 should be reserved for exceptional performance in the given criterion.

Then, also output the total score of a solution, out of the maximum of 30 points.

Repeat for All Solutions: Follow the steps above for each solution presented in the input text.

Objective: This structured analysis will facilitate a comprehensive comparison of the solutions, highlighting their strengths and areas for improvement based on the defined criteria.

In your output, please ensure that each solution is separated by "\n---\n".
"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



### Extracted Solutions

**1. Smart Water Avatar**
- **Solved Problem:** Inefficiency and waste due to appliances using standard settings regardless of water quality.
- **Solution Description:** An AI-driven platform optimizing water usage across home appliances by learning user habits and adapting to real-time water quality.
- **Impact:** Enhances water and energy efficiency, reducing environmental footprint and operational costs.
- **Technology Used:** Advanced AI, machine learning, integration with IoT devices, robust data privacy measures.
- **Sources:** Inspired by e2ip technologies.

**2. Hydro-Responsive Appliances**
- **Solved Problem:** Resource wastage from lack of real-time water quality adjustment in appliances.
- **Solution Description:** Appliances with material science sensors that adjust operations based on water quality changes without electronic inputs.
- **Impact:** Makes appliances adaptable to varying water qualities, enhancing environmental sustainability.
- **Tec

In [ ]:
# Pick the best three 

user_prompt5 = f"""
Extract and rank the solutions based on their overall ratings from the provided data: {initial_rating}.

Identify the top three solutions with the highest overall ratings out of 30 and sort them in descending order of their ratings.

For each of these top solutions, provide detailed information based on the content from: {summary_solutions}. Ensure comprehensive coverage of the points listed below:

- "Name": The solution's name, limited to a maximum of 5 words.
- "Solved Problem": A concise three-sentence summary of the problem addressed by the solution (minimum 50 words).
- "Solution": A brief summary of the solution, including its mechanism, in 4 to 5 sentences.
- "Impact": Explanation of how the solution addresses the specific problem, in 2 to 3 sentences.
- "Technology": Detailed description of the technologies used, including their application, aimed at a CTO with an engineering background for implementation purposes, in 4-5 sentences.
- "Sources": Mention any specific inspiration sources from {summary_list}, limited to 1 sentence.

Additionally, highlight the rating criteria where each solution excels.

Please separate each solution's details with "---".

"""

# Call the GPT model to generate the completion
completion = openai.chat.completions.create(
    model="gpt-4-0125-preview",
    messages=[
    {"role": "system", "content": system_prompt_2},
    {"role": "user", "content": user_prompt5}
    ],
    temperature=0.7,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

# Correctly access the generated text from the completion response
generated_text = completion.choices[0].message.content

# Split the generated text into individual solutions based on your specified separator ("---")
solutions_list = generated_text.split("---")

# Initialize an empty dictionary to hold your final solutions
final_solutions = {}

# Populate the dictionary with distinct solutions
for index, solution_text in enumerate(solutions_list, start=1):
    if solution_text.strip():  # Ensure the string is not just whitespace
        final_solutions[index] = solution_text.strip()

# Ensure only the top 3 solutions are kept if more were accidentally generated
final_solutions = dict(list(final_solutions.items())[:3])

# 'final_solutions' now contains up to three distinct items as requested

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


### Smart Water Avatar

- **Name**: Smart Water Avatar
- **Solved Problem**: This solution addresses the inefficiency of home appliances using uniform settings, regardless of varying water quality. It mitigates the waste and increased operational costs associated with non-optimized water and energy use in household appliances, catering to the growing demand for smarter, more sustainable home solutions.
- **Solution**: The Smart Water Avatar is an AI-driven platform that dynamically optimizes water usage across household appliances by learning user habits and adapting to changes in real-time water quality. It suggests efficiency improvements and maintenance schedules for appliances, such as washing machines and garden irrigation systems, enhancing overall sustainability.
- **Impact**: By dynamically adjusting appliance operations based on water quality and user habits, this solution significantly reduces water and energy consumption, leading to a smaller environmental footprint and lowe

In [ ]:
# Integrate new mergeflow data for the solutions 

# Now, based on soultions found, we need more info from Mergeflow


boolean_queries = {}

for index, solution_name in final_solutions.items():
    user_prompt6 = f"""
    
    Create a boolean query based on the Name of the solution from {solution_name}.
    
    Please use the following as an example and follow the format: 
    
    Example: for Factory Blockchain Protocol, this would be:

    "Factory Blockchain Protocol" OR "FBP" OR "blockchain for manufacturing" OR "blockchain in factories" OR "blockchain in production" OR "blockchain in supply chain" OR "blockchain in logistics" OR "blockchain in industry" OR "blockchain in industrial processes" AND factory OR manufacturing OR production OR supply chain OR logistics OR industry OR industrial processes
    

    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
        {"role": "system", "content": system_prompt_2},
        {"role": "user", "content": user_prompt6}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )

    # Extracting the content of the message from the first choice in the choices list
    query_content = completion.choices[0].message.content
    
    # Save the generated boolean query for the current solution
    boolean_queries[index] = query_content

    # Print the generated boolean query for the current solution
    print(query_content)
    print("\n----\n")
    

    

"Smart Water Avatar" OR "SWA" OR "AI water optimization" OR "AI in home appliances" OR "smart water management" OR "intelligent water use" OR "water efficiency AI" OR "AI water saving" OR "AI for sustainable water" AND home appliances OR washing machines OR dishwashers OR garden irrigation OR water softeners OR water purifiers OR household sustainability OR energy consumption

----

"AI Water Concierge" OR "AWC" OR "AI for water management" OR "AI in water conservation" OR "AI in water usage" OR "AI for water efficiency" OR "AI in water systems" OR "AI in home water management" AND water management OR water conservation OR water usage OR water efficiency OR home water systems OR water treatment OR sustainable water use

----

"Decentralized Water Network" OR "DWN" OR "blockchain in water management" OR "blockchain for water sharing" OR "blockchain in water treatment" OR "blockchain in community water systems" OR "blockchain in water sustainability" OR "blockchain in water resources" AN

In [ ]:
len(boolean_queries)

3

In [ ]:
# the Mergeflow API call

dataset = '&sp=2655' # access dataset from mergeflow - in this case science publications
rows = str(3) # first 3 docs only

# Iterate through each boolean query
for solution, query in boolean_queries.items():
    # Construct the API request URL
    url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

    # Define the headers to be sent with the request
    headers = {
        'MergeflowNet-Api-Auth-Key': mergeflow_api_key
    }

    # Send the POST request
    response = requests.post(url, headers=headers)

    mergeflow_docs = []

    # Check the response status code and content
    if response.status_code == 200:
        mergeflow_response_json = response.json()
        
        # Extract and store documents information
        for document in mergeflow_response_json['Documents']:
            current_doc = {
                'title': document['Title'],
                'date': document['Date'],
                'url': document['Url'],
                'content': document['Content']
            }
            
            mergeflow_docs.append(current_doc)

    # Process or print your results here
    # For example, print titles of documents for this query
    print(f"Results for {solution}:")
    for doc in mergeflow_docs:
        print(doc['title'])
    print("\n----\n")      


Results for 1:

----

Results for 2:

----

Results for 3:

----

